In [28]:
  
import argparse
import copy
import os
import os.path as osp
import time

import mmcv
import torch
from mmcv.runner import init_dist
from mmcv.utils import Config, DictAction, get_git_hash

from mmseg import __version__
from mmseg.apis import set_random_seed, train_segmentor
from mmseg.datasets import build_dataset, build_dataloader
from mmseg.models import build_segmentor
from mmseg.utils import collect_env, get_root_logger

In [8]:
def parse_args():
    parser = argparse.ArgumentParser(description='Train a segmentor')
    parser.add_argument('config', help='train config file path')
    parser.add_argument('--work-dir', help='the dir to save logs and models')
    parser.add_argument(
        '--load-from', help='the checkpoint file to load weights from')
    parser.add_argument(
        '--resume-from', help='the checkpoint file to resume from')
    parser.add_argument(
        '--no-validate',
        action='store_true',
        help='whether not to evaluate the checkpoint during training')
    group_gpus = parser.add_mutually_exclusive_group()
    group_gpus.add_argument(
        '--gpus',
        type=int,
        help='number of gpus to use '
        '(only applicable to non-distributed training)')
    group_gpus.add_argument(
        '--gpu-ids',
        type=int,
        nargs='+',
        help='ids of gpus to use '
        '(only applicable to non-distributed training)')
    parser.add_argument('--seed', type=int, default=None, help='random seed')
    parser.add_argument(
        '--deterministic',
        action='store_true',
        help='whether to set deterministic options for CUDNN backend.')
    parser.add_argument(
        '--options', nargs='+', action=DictAction, help='custom options')
    parser.add_argument(
        '--launcher',
        choices=['none', 'pytorch', 'slurm', 'mpi'],
        default='none',
        help='job launcher')
    parser.add_argument('--local_rank', type=int, default=0)
    args = parser.parse_args()
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args.local_rank)

    return args
args = parse_args()

usage: ipykernel_launcher.py [-h] [--work-dir WORK_DIR] [--load-from LOAD_FROM] [--resume-from RESUME_FROM]
                             [--no-validate] [--gpus GPUS | --gpu-ids GPU_IDS [GPU_IDS ...]] [--seed SEED]
                             [--deterministic] [--options OPTIONS [OPTIONS ...]]
                             [--launcher {none,pytorch,slurm,mpi}] [--local_rank LOCAL_RANK]
                             config
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [51]:
parser = argparse.ArgumentParser(description='Train a segmentor')
parser.add_argument('--config', help='train config file path')
group_gpus = parser.add_mutually_exclusive_group()
group_gpus.add_argument(
    '--gpus',
    type=int,
    help='number of gpus to use '
    '(only applicable to non-distributed training)')
group_gpus.add_argument(
    '--gpu-ids',
    type=int,
    nargs='+',
    help='ids of gpus to use '
    '(only applicable to non-distributed training)')
args = parser.parse_args(args=['--config', "configs/unet/deeplabv3_unet_s5-d16_64x64_40k_particle_detection.py",
                              '--gpus', "1", 
                              
                               ]
                               )

In [55]:
args.gpu_ids = 1

args


Namespace(config='configs/unet/deeplabv3_unet_s5-d16_64x64_40k_particle_detection.py', gpu_ids=1, gpus=1)

In [59]:
cfg = Config.fromfile(args.config)

In [60]:
cfg

Config (path: configs/unet/deeplabv3_unet_s5-d16_64x64_40k_particle_detection.py): {'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'model': {'type': 'EncoderDecoder', 'pretrained': None, 'backbone': {'type': 'UNet', 'in_channels': 3, 'base_channels': 64, 'num_stages': 5, 'strides': (1, 1, 1, 1, 1), 'enc_num_convs': (2, 2, 2, 2, 2), 'dec_num_convs': (2, 2, 2, 2), 'downsamples': (True, True, True, True), 'enc_dilations': (1, 1, 1, 1, 1), 'dec_dilations': (1, 1, 1, 1), 'with_cp': False, 'conv_cfg': None, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'act_cfg': {'type': 'ReLU'}, 'upsample_cfg': {'type': 'InterpConv'}, 'norm_eval': False}, 'decode_head': {'type': 'ASPPHead', 'in_channels': 64, 'in_index': 4, 'channels': 16, 'dilations': (1, 12, 24, 36), 'dropout_ratio': 0.1, 'num_classes': 2, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'align_corners': False, 'loss_decode': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}}, 'auxiliary_head': {'type': 'F

In [57]:
datasets = [build_dataset(cfg.data.train)]

2021-04-17 11:09:24,786 - mmseg - INFO - Loaded 5837 images
INFO:mmseg:Loaded 5837 images


In [66]:
distributed = False
seed = 0
data_loaders = [
        build_dataloader(
            ds,
            cfg.data.samples_per_gpu,
            cfg.data.workers_per_gpu,
            # cfg.gpus will be ignored if distributed
            1,#len(cfg.gpu_ids),
            dist=distributed,
            seed=seed,
            drop_last=True) for ds in datasets
    ]

In [75]:
data_loaders.__getitem__(0)

In [83]:
datasets[0].__getitem__(0)['img'].data.shape

torch.Size([3, 512, 1024])

In [90]:
import numpy as np
np.unique(datasets[0].__getitem__(0)['gt_semantic_seg'].data.detach().numpy())

array([  0,   1, 255])

In [95]:
cfg.data.train

{'type': 'CustomDataset',
 'data_root': '/home/shirakawa/projects/pytorch_particles_detection/dataset_sample/mmseg_dataset/',
 'img_dir': 'img_dir/train',
 'ann_dir': 'ann_dir/train',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations'},
  {'type': 'Resize', 'img_scale': (512, 32), 'ratio_range': (0.5, 2.0)},
  {'type': 'RandomCrop', 'crop_size': (512, 1024), 'cat_max_ratio': 0.75},
  {'type': 'RandomFlip', 'prob': 0.5},
  {'type': 'PhotoMetricDistortion'},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'Pad', 'size': (512, 1024), 'pad_val': 0, 'seg_pad_val': 255},
  {'type': 'DefaultFormatBundle'},
  {'type': 'Collect', 'keys': ['img', 'gt_semantic_seg']}]}